In [ ]:
import os
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from exp_spec_info import *
from plot_info import *
from select_data import *

In [ ]:
# Processed pickle paths
extended_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extended_data.pkl"
median_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\median_data.pkl"

# Plot output directory
chap_3_plot_dir = "C:\\Users\\dosre\\Desktop\\MSCthesis\\thesis\\body_chapters\\chap_3\\images"
os.makedirs(chap_3_plot_dir, exist_ok=True)
chap_6_plot_dir = "C:\\Users\\dosre\\Desktop\\MSCthesis\\thesis\\body_chapters\\chap_6\\images\\anomalous"
os.makedirs(chap_6_plot_dir, exist_ok=True)

In [ ]:
# Load data
extended_data = pd.read_pickle(extended_data_path)
median_data = pd.read_pickle(median_data_path)

##### 3 Generate Ideal and Non-Ideal Example Cases

In [ ]:
experiments_spec_to_plot = [
    ("Bump_2911", "unprecond", 20),
    ("ns3Da", "unprecond", 20)
]

for matrix, setup, restart_param in experiments_spec_to_plot:

    sub_data = df_sel_setup_matrix_restart(
        extended_data, setup, matrix, restart_param
    )

    fig, ax = plt.subplots(figsize=(6.4, 3.2))

    for fp_solver in FP_SOLVERS:
        plot_data = sub_data[sub_data["solver"] == fp_solver]
        plot_exp_iters_conv_traj(
            ax,
            plot_data,
            N_EXPERIMENT_ITERATIONS,
            fp_solver,
            SOLVER_CLR_DICT[fp_solver]     
        )

    ax.legend()
    ax.grid()
    ax.set_ylabel("$\\|b-Ax_{i}\\|_{2}/\\|b-Ax_{0}\\|_{2}$")
    ax.set_xlabel("Inner Iteration")
    
    fig.suptitle(
        f"{matrix} {SETUP_NAME_MAPPING[setup]} GMRES({restart_param})"
    )
    
    plt.savefig(
        os.path.join(chap_3_plot_dir, f"{setup}_{matrix}_{restart_param}.png"),
        bbox_inches="tight"
    )

    plt.show()
    plt.close()
    

##### 7.5 Anomalous Occurrences

In [ ]:
experiments_spec_to_plot = [
    ("af_0_k101", "unprecond", 100),
    ("rajat31", "unprecond", 100)
]

for matrix, setup, restart_param in experiments_spec_to_plot:

    fig, axs = plt.subplots(2, 1, figsize=(6.4, 5.6))
    ax1, ax2 = axs

    extended_sub_data = df_sel_setup_matrix_restart(
        extended_data, setup, matrix, restart_param
    )
    median_sub_data = df_sel_setup_matrix_restart(
        median_data, setup, matrix, restart_param
    )

    for fp_solver in FP_SOLVERS:
        solver_exp_iteration_data = extended_sub_data[
            extended_sub_data["solver"] == fp_solver
        ]
        plot_exp_iters_conv_traj(
            ax1,
            solver_exp_iteration_data,
            N_EXPERIMENT_ITERATIONS,
            fp_solver,
            SOLVER_CLR_DICT[fp_solver]                    
        )

    for gmres_m_solver in GMRES_M_SOLVERS:
        solver_exp_iteration_data = extended_sub_data[
            extended_sub_data["solver"] == gmres_m_solver
        ]
        plot_exp_iters_conv_traj(
            ax2,
            solver_exp_iteration_data,
            N_EXPERIMENT_ITERATIONS,
            gmres_m_solver,
            SOLVER_CLR_DICT[gmres_m_solver]                    
        )

    for ax in [ax1, ax2]:
        ax.set_xlim(
            0,
            np.nanmax(np.hstack([0, extended_sub_data["inner_iters"].to_numpy()]))
        )
        ax.set_ylabel("$\\|b-Ax_i\\|_2/\\|b-Ax_0\\|_2$")
        ax.legend()
        ax.grid()
    ax1.set_xticklabels([])
    ax2.set_xlabel("Inner Iteration")

    fig.suptitle(
        f"{matrix} {SETUP_NAME_MAPPING[setup]} GMRES({restart_param})"
    )
    
    fig.tight_layout()

    plt.savefig(
        os.path.join(chap_6_plot_dir, f"{setup}_{matrix}_{restart_param}.png"),
        bbox_inches="tight"
    )
    plt.show()

    plt.close()